In [1]:
import pandas as pd

# Load the CSV file
file_path = '/Users/sheillapurwandiary/spurwand/BIPM/Semester2/001EABD/portfolio/Giki_Big-Data/Data/cleaned_news_articles.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
data.head()

,source,url,date,time,title,body,summary,keywords,image_url,clean_body
0,CNBC,https://www.cnbc.com/2024/06/24/southeast-asia...,24.06.24,02:51:46 UTC,Southeast Asia is a top choice for firms diver...,"Skyline, Ho Chi Minh City, Vietnam John Harper...","Skyline, Ho Chi Minh City, Vietnam John Harper...","['supply', 'indonesia', 'manufacturing', 'chin...",https://image.cnbcfm.com/api/v1/image/10146887...,skylin ho chi minh citi vietnam john harper ph...
1,CNBC,https://www.cnbc.com/2024/06/24/asia-stock-mar...,24.06.24,03:12:12 UTC,Asia-Pacific markets slip as investors await A...,Asia-Pacific markets mostly fell on Monday as ...,Asia-Pacific markets mostly fell on Monday as ...,"['markets', 'later', 'bank', 'rba', 'central',...",https://image.cnbcfm.com/api/v1/image/10730059...,asiapacif market most fell monday investor awa...
2,CNBC,https://www.cnbc.com/2024/06/24/paris-2024-sin...,24.06.24,00:45:35 UTC,Singaporean swimmer Toh Wei Soong hopes for a ...,"Toh, a Singaporean freestyle and butterfly swi...","Toh, a Singaporean freestyle and butterfly swi...","['paralympic', 'athletes', 'swimmer', 'hopes',...",https://image.cnbcfm.com/api/v1/image/10742765...,toh singaporean freestyl butterfli swimmer set...
3,Financial Times,https://www.ft.com/content/768481a1-60a9-45a3-...,24.06.24,01:37:03 UTC,Live news: Prudential shares rise after announ...,Try unlimited access\n\nOnly 1 € for 4 weeks\n...,Try unlimited accessOnly 1 € for 4 weeksThen 6...,"['trial', '2bn', 'monthcomplete', 'prudential'...",https://www.ft.com/__origami/service/image/v2/...,tri unlimit access eur week eur per month comp...
4,Fortune,https://fortune.com/well/article/best-time-of-...,24.06.24,00:00:00 UTC,"The best time of day to exercise, according to...","Claire Zulkey, a 44-year-old Chicago-area free...",Consider your fitness goalsA small study out o...,"['science', 'morning', 'workouts', 'night', 's...",https://fortune.com/img-assets/wp-content/uplo...,clair zulkey yearold chicagoarea freelanc writ...


In [5]:
import ssl
import nltk

# Bypass SSL verification
ssl._create_default_https_context = ssl._create_unverified_context

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sheillapurwandiary/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sheillapurwandiary/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sheillapurwandiary/nltk_data...


True

In [7]:
import gensim
from gensim import corpora
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from gensim.parsing.preprocessing import preprocess_string

# Custom preprocessing function
def preprocess(text):
    # Use Gensim's simple preprocessing
    text = preprocess_string(text.lower())
    # Remove stopwords using sklearn's stopword list
    text = [word for word in text if word not in ENGLISH_STOP_WORDS and word.isalpha()]
    return text

# Apply preprocessing
texts = data['clean_body'].dropna().tolist()
processed_texts = [preprocess(text) for text in texts]


In [8]:
# Create a dictionary and corpus
dictionary = corpora.Dictionary(processed_texts)
corpus = [dictionary.doc2bow(text) for text in processed_texts]

# Save dictionary and corpus for future use
dictionary.save('/Users/sheillapurwandiary/spurwand/BIPM/Semester2/001EABD/portfolio/Giki_Big-Data/lda_dictionary.dict')
gensim.corpora.MmCorpus.serialize('/Users/sheillapurwandiary/spurwand/BIPM/Semester2/001EABD/portfolio/Giki_Big-Data/lda_corpus.mm', corpus)


In [10]:
from gensim.models import LdaModel

# Set the number of topics
num_topics = 5

# Fit the LDA model
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, random_state=42, passes=10)

# Save the LDA model
lda_model.save('/Users/sheillapurwandiary/spurwand/BIPM/Semester2/001EABD/portfolio/Giki_Big-Data/lda_model.model')

# Display the topics
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)


(0, '0.012*"chip" + 0.012*"china" + 0.010*"said" + 0.010*"taiwan" + 0.008*"year" + 0.007*"byte" + 0.007*"market" + 0.006*"chine" + 0.005*"stock" + 0.005*"like"')
(1, '0.021*"bank" + 0.011*"said" + 0.011*"famili" + 0.006*"board" + 0.006*"architect" + 0.005*"year" + 0.005*"build" + 0.005*"kenya" + 0.005*"govern" + 0.005*"price"')
(2, '0.020*"bank" + 0.011*"rate" + 0.010*"year" + 0.008*"yen" + 0.007*"inflat" + 0.007*"capit" + 0.007*"japan" + 0.006*"billion" + 0.006*"dollar" + 0.006*"polici"')
(3, '0.009*"sleep" + 0.009*"said" + 0.009*"exerci" + 0.008*"time" + 0.008*"narcolepsi" + 0.006*"work" + 0.005*"condit" + 0.005*"toh" + 0.004*"peopl" + 0.004*"court"')
(4, '0.009*"china" + 0.008*"said" + 0.007*"compani" + 0.007*"duti" + 0.007*"year" + 0.006*"chine" + 0.006*"market" + 0.006*"busi" + 0.005*"data" + 0.005*"firm"')


In [13]:
import pyLDAvis
import pyLDAvis.gensim_models

# Prepare the visualization
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary, sort_topics=False)

# Save the visualization to an HTML file
pyLDAvis.save_html(lda_display, '/Users/sheillapurwandiary/spurwand/BIPM/Semester2/001EABD/portfolio/Giki_Big-Data/lda_visualization.html')

